# The News Hour 
## Estimating the welfare impact of television news

### by Matthew Baker and Lisa George

In our paper, which you can find [at SSRN](http://papers.ssrn.com/sol3/papers.cfm?abstract_id=2694687), we describe what turns out to be a rather complex model of television news. This model includes several features, including:

1. A viewership model with nested multinomial logit demand,
2. A log-linear price model, 
3. Strategic interactions among stations with the aim of getting the most advertising revenues: Stations choose program menus to maximize their average nightly advertising revenues, which depend upon viewership shares and programming choices. 

Since there are a lot of moving parts - that we ourselves have to keep track of in details - we thought a notebook detailing various aspects of estimation would be a good idea, particularly if anyone else wanted to use some of the methods! In this first notebook, I will discuss data setup. 

# Data Setup

The first thing to do, of course, is read in the data and render it in a manageable form. We need to revise settings and all that as well. We focus on the 2010 data, where our `sample` variable marks useable observations. We also exclude observations for which the variable `h` is not equal to twenty. This means that we are excluding observations from the twentieth hour of the day (e.g, 8 o'clock) because that is into prime time. 


In [55]:
import ipystata
import os

CWD = os.getcwd()
print(CWD)

C:\Users\mjbaker\Documents\GitHub\NewsHour


There has to be a better way to do this than just running the above command and cutting and pasting...

In [56]:
%%stata
clear all
cd C:\Users\mjbaker\Documents\GitHub\NewsHour


C:\Users\mjbaker\Documents\GitHub\NewsHour


In [57]:
%%stata
disp c(pwd)


C:\Users\mjbaker\Documents\GitHub\NewsHour


In [58]:
%%stata

clear all
set matsize 1000
use "Data\NielsenKantarPanel.dta" if year == 2010 & sample == 1 & h != 20

Now, we make a variable marking all of our timeslots. Observe that the time slots are marked by hour, and then whether or not they are the first or second half of the hour. Accordingly:

In [59]:
%%stata

gen     timeslot = 1 if h == 16 & half == 1
replace timeslot = 2 if h == 16 & half == 2
replace timeslot = 3 if h == 17 & half == 1
replace timeslot = 4 if h == 17 & half == 2
replace timeslot = 5 if h == 18 & half == 1
replace timeslot = 6 if h == 18 & half == 2
replace timeslot = 7 if h == 19 & half == 1
replace timeslot = 8 if h == 19 & half == 2


(2417240 missing values generated)
(345320 real changes made)
(345320 real changes made)
(345320 real changes made)
(345320 real changes made)
(345320 real changes made)
(345320 real changes made)
(345320 real changes made)


Clean up the population variables so that they don't vary in each region or so they aren't missing for some observations:

In [60]:
%%stata
bysort NDMACode: egen TotalPop1 = max(TotalPop)
replace TotalPop=TotalPop1


(2585760 real changes made)


Another problem we face with the data is missing values for viewership under some level. Accordingly, we just replace these values with one-half the minimum reported value for each viewer. 

In [61]:
%%stata
egen double minviewers=min(viewers), by(NDMAC)
replace viewers=1/2*minviewers if viewers==. & affil=="" 



(1519397 real changes made)


The next step is to employ an adjustment factor - which really just serves the purpose of not letting shares go above unity! A simple fix to this problem is to just suppose that each household can watch two shows per half-hour block, which effectively multiplies the population by two. From here on out, `Mpop` will function as our effective population variable. 

In [62]:
%%stata
scalar adjfactor=2
egen double Mpop=mean(TotalPop), by(NDMACode)
replace Mpop=Mpop*adjfactor
label var Mpop "Mean TotalPop by DMA"


(2762560 real changes made)


An alternative measure of viewership, which is in some sense more accurate, is the American Community Survey reported households in the DMA. We can use this = along with the adjustment factor - to compute effective ACS eyeballs. In some very small markets, we get funny shares, so it is good to get rid of markets where this happens as well. 

Schematic:
1. Multiply ACS by effective ACS adjustment factor
2. Generate viewership shares
3. Mark occurrence of excessive shares
4. Mark markets with an occurrence of an excessive share
5. Mark these markets for exclusion.

In [63]:
%%stata
replace ACS_HH = ACS_HH*adjfactor
gen double si = viewers/ACS_HH                                  
egen double S = sum(si), by(NDMACode dow timeslot week)          
egen double maxS = max(S), by(NDMACode)                          
replace sample = 0 if maxS > 1                                     


(2762560 real changes made)
(27817 missing values generated)
(103040 real changes made)


Update the data:

In [64]:
%%stata
keep if sample == 1


(103040 observations deleted)


Categorical variables marking different types of programming, according to the classifications that Lisa did. Our four categories will be:

1. Local news (`lnews`) - requires expanding the existing definition a bit.
2. National news (`nnews`)
3. Other local programming - referred to as "entertainment" in the paper (`otherl`)
4. Other cable programming (`otherc`)

In [65]:
%%stata
rename lnews l_all
gen lnews    = cond(l_all==1 & bc==1,1,0)
gen cablenews= cond(Nstat=="CNN" | Nstat=="BBCA" | Nstat=="HLN" | Nstat=="WGN" | Nstat=="FXNC",1,0)
gen nnews    = cond(news==1 & l_all==0 & (bc == 1 |cablenews == 1),1,0)
gen otherl   = cond(news==0 & bc==1,1,0)
gen otherc   = cond(bc==0 & lnews==0 & nnews==0 & otherl==0,1,0)


> )


Mark price data and mark and viewership data. Price data is defined as data for which we have a price per second (`pps != 0`), while viewership data is data for which we have a viewership shape (`si != .`). We also take this opportunity to drop out the spottier data on the first two time periods in our sample. 

In [66]:
%%stata
gen pricedata = cond(pps != ., 1, 0)
gen viewdata  = cond(si != ., 1, 0)
replace viewdata = 0 if timeslot ==1 | timeslot ==2


(656167 real changes made)


Now, we will start assembling station-level variables by time slot across the four weeks of daily data in our data set. This focuses attention on average viewership over the time period (which is the relevant thing to look at for programming decisions), and also reduces our data to a manageable size! Before doing this, we pare down the data to our viewership data, and then create a station-level unique identifier number.

In [67]:
%%stata
keep if viewdata


(683382 observations deleted)


Schematic for the following variables: 

1. Station-level numeric Id
2. Mean viewership share for weekdays over sweeps month.
3. Mean price-per-second for weekdays over sweeps month.
4. Programming type - defined as a share of days broadcasting each type of programming. 

In [68]:
%%stata
egen stationid = group(NDMAName Nstat)

bysort stationid timeslot : egen double meansi    =mean(si)
bysort stationid timeslot : egen double meanpps   =mean(pps)
bysort stationid timeslot : egen double meanlnews =mean(lnews)
bysort stationid timeslot : egen double meanotherl=mean(otherl)
bysort stationid timeslot : egen double meannnews =mean(nnews)
bysort stationid timeslot : egen double meanotherc=mean(otherc)
bysort stationid timeslot :  gen keep =_n==_N

keep if keep


(1903095 missing values generated)
(1876427 observations deleted)


Pare down the data so that we keep only what we need - it still is a huge data set, after all!


In [69]:
%%stata
keep NDMAName NDMACode Nstat stationid timeslot affiliation local_station ACS_HH ACS_HHBlack ACS_HHHispanic ACS_MedianHHIncome ACS_HHWhite ACS_MedianAge Mpop meansi meanpps meanlnews meanotherl meannnews meanotherc


> HHispanic ACS_MedianHHIncome ACS_HHWhite ACS_MedianAge Mpop meansi meanpps meanlnews meanotherl me
> annnews meanotherc


Sort resulting data, fill in missings, and set panel identifiers:

In [70]:
%%stata
sort NDMACode stationid timeslot
tsset stationid timeslot
tsfill, full


       panel variable:  stationid (unbalanced)
        time variable:  timeslot, 3 to 8, but with gaps
                delta:  1 unit


THe next bit of code back fills non-varying demographic variables for each observation. It loops over time periods and puts, for example, population data wherever it is missing at our panel-time unit. 

In [71]:
%%stata
local stringReplace NDMAName Nstat affiliation
local numberReplace stationid timeslot local_station ACS_HH ACS_HHBlack ACS_HHHispanic ACS_MedianHHIncome ACS_HHWhite ACS_MedianAge Mpop NDMACode

forvalues i=3/8 {
    foreach word of local stringReplace {
        bysort stationid: replace `word'=`word'[_n+1] if `word'==""
        bysort stationid: replace `word'=`word'[_n-1] if `word'==""
    }
    foreach word of local numberReplace {
        bysort stationid: replace `word'=`word'[_n+1] if `word'==.
        bysort stationid: replace `word'=`word'[_n-1] if `word'==.
    }
}


> HIncome ACS_HHWhite ACS_MedianAge Mpop NDMACode(13 real changes made)
(1 real change made)
(13 real changes made)
(1 real change made)
(13 real changes made)
(1 real change made)
(0 real changes made)
(0 real changes made)
(0 real changes made)
(0 real changes made)
(13 real changes made)
(1 real change made)
(13 real changes made)
(1 real change made)
(13 real changes made)
(1 real change made)
(13 real changes made)
(1 real change made)
(13 real changes made)
(1 real change made)
(13 real changes made)
(1 real change made)
(13 real changes made)
(1 real change made)
(13 real changes made)
(1 real change made)
(13 real changes made)
(1 real change made)
(6 real changes made)
(0 real changes made)
(6 real changes made)
(0 real changes made)
(6 real changes made)
(0 real changes made)
(0 real changes made)
(0 real changes made)
(0 real changes made)
(0 real changes made)
(6 real changes made)
(0 real changes made)
(6 real changes made)
(0 real changes made)
(6 real changes made)
(0 r

We have some missing shares here and there, so we can fill these in without too much trouble. The best thing to do is interpolate over logs, as in this way we ensure that results are always positive - we can't have negative shares when we go to the form modelling!

In [72]:
%%stata

gen double lnsi = ln(meansi)
bysort stationid: ipolate lnsi timeslot, gen(lnsid) epolate
replace lnsi = lnsid if lnsi==.
replace meansi = exp(lnsi) if meansi==.
drop lnsid


(21 missing values generated)
(21 real changes made)
(21 real changes made)


The next block of code interpolates missing programming types where they are missing - there aren't a lot of places where this is the case, so there is virtually zero harm done. 

We then replace the most common programming type with a dummy for the analysis. Once again, stations almost never change viewership patterns during the week, but there are a handful of stations that do. 

In [73]:
%%stata
local AverageReplace meanlnews meanotherl meannnews meanotherc meanpps

foreach word of local AverageReplace {
    bysort stationid: ipolate `word' timeslot, gen(holder) epolate
    replace `word'=holder if `word'==.
    drop holder
}

gen lnews  = meanlnews > .5
gen nnews  = meannnews > .5
gen otherl = lnews == 0 & nnews == 0 & local_station
gen otherc = (lnews == 0 & nnews == 0 & otherl == 0)

drop meanlnews meanotherl meannnews meanotherc

rename meansi si
rename meanpps pps


(21 real changes made)
(21 real changes made)
(21 real changes made)
(21 real changes made)
(95994 missing values generated)
(0 real changes made)


Now that we have average shares and programming types per station per time period, we can start forming aggregates, which will figure into estimation when we start using the nested multinomial logit. Here, we compute shares by time period and programming type for each market, and also generate total viewership shares (and not viewing shares). We also make a variable called `owngroupshare` which returns each stations share within group of whatever type of broadcast they happen to be showing. 

In [74]:
%%stata
bysort NDMAC timeslot: egen double slnews=total(si*lnews)
bysort NDMAC timeslot: egen double snnews=total(si*nnews)
bysort NDMAC timeslot: egen double sotherl=total(si*otherl)
bysort NDMAC timeslot: egen double sotherc=total(si*otherc)

bysort NDMAC timeslot: egen double stotal=total(si)
gen double so=1-stotal

gen double owngroupshare=lnews*slnews+nnews*snnews+otherl*sotherl+otherc*sotherc

Logs and interaction terms, which also figure into estimation. Coincidentally, we generate these as doubles because they are often somewhat small, and we like to keep significant digits in making small predictions. 

In [75]:
%%stata
gen double ln_swg = ln(si/owngroupshare) 

gen double ln_swgXslnews=ln_swg*lnews   
gen double ln_swgXsnnews=ln_swg*nnews    
gen double ln_swgXsotherl=ln_swg*otherl

Station-affiliation dummies:

In [76]:
%%stata
gen NBC=(aff=="NBC")
gen CBS=(aff=="CBS")
gen ABC=(aff=="ABC")
gen FOX=(aff=="FOX")
gen CW= (aff=="CW")
gen TEL=(aff=="TEL")
gen UNI=(aff=="UNI")
gen AZA=(aff=="AZA")
gen MNT=(aff=="MNT")
gen PBS=(aff=="PBS")
gen TLF=(aff=="TLF")

Some additional variables of interest, including a market-time identifier, and a market identifier (both are needed for our random effects scheme). Also, a time id, and a dependent variable. We then save it all as our working data set.

In [77]:
%%stata
egen market=group(NDMACode)

egen mt=group(NDMACode timeslot)	

sort market mt stationid

tab timeslot, gen(timed)
tab NDMACode, gen(markket)

gen double dln=ln(si)-ln(so)
gen double lnpps=ln(pps)

gen double lnsiXnnews=lnsi*nnews
gen double lnsiXotherl=lnsi*otherl

sort market stationid mt

save "Data\AveragedData.dta", replace


   timeslot |      Freq.     Percent        Cum.
------------+-----------------------------------
          3 |     16,622       16.67       16.67
          4 |     16,622       16.67       33.33
          5 |     16,622       16.67       50.00
          6 |     16,622       16.67       66.67
          7 |     16,622       16.67       83.33
          8 |     16,622       16.67      100.00
------------+-----------------------------------
      Total |     99,732      100.00

   NDMACode |      Freq.     Percent        Cum.
------------+-----------------------------------
        500 |        402        0.40        0.40
        501 |        744        0.75        1.15
        502 |        456        0.46        1.61
        503 |        474        0.48        2.08
        504 |        750        0.75        2.83
        505 |        612        0.61        3.45
        506 |        654        0.66        4.10
        507 |        438        0.44        4.54
        508 |        642      

To the next workbook!